# Part 3, Multilayer Perceptron Neural Network

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sys
import h5py
import pickle
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from scipy.sparse import csr_matrix, hstack
from sklearn.cross_validation import KFold, train_test_split
from keras.models import Sequential
from keras.models import save_model, load_model
from keras.layers.advanced_activations import PReLU
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
import matplotlib.patches as mpatches
import time

%matplotlib inline

/home/csc/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/csc/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using TensorFlow backend.


In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
'''gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))'''
K.set_session(sess)

In [5]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [6]:
train = pd.read_csv('train.csv')

cat_names = [c for c in train.columns if 'cat' in c]

train = pd.get_dummies(data=train, columns=cat_names)

features = [x for x in train.columns if x not in ['id','loss']]

train_x = np.array(train[features])

ntrain = train_x.shape[0]

train_y = np.log(train['loss'] + 200)

print (train_x.shape)
print (train_y.shape)
train_x.shape, train_y.shape

(188318, 1153)
(188318,)


((188318, 1153), (188318,))

In [7]:
def cross_validate_mlp(mlp_func, nfolds=3):
    folds = KFold(len(train_y), n_folds=nfolds, shuffle=True, random_state = 31337)
    val_scores = np.zeros((nfolds,))
    for k,(train_index, test_index) in enumerate(folds):
        xtr = train_x[train_index]
        ytr = train_y[train_index]
        xte = train_x[test_index]
        yte = train_y[test_index]
        mlp = mlp_func()
        early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        fit = mlp.fit(xtr, ytr, validation_split=0.2, batch_size=128, 
                      nb_epoch=30, verbose=1, callbacks=[early_stopping])
        pred = mlp.predict(xte, batch_size=256)
        score = mean_absolute_error(yte, pred)
        val_scores[k] += score
        print ('Fold {}, MAE: {}'.format(k, score))
    avg_score = np.sum(val_scores) / float(nfolds)
    print ('{}-fold CV score: {}'.format(nfolds, avg_score))
    return avg_score

In [8]:
xtr, xte, ytr, yte = train_test_split(train_x, train_y, test_size=0.25, random_state=31337)

<img src="http://cdn.rawgit.com/dnkirill/allstate_capstone/master/images/mlp3.svg"></td>

In [9]:
import keras.backend as K
from sklearn.metrics import mean_absolute_error, make_scorer

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))

def xg_eval_mae(yhat, dtrain):
    y = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(y), np.exp(yhat))

def mae_score(y_true, y_pred):
    return mean_absolute_error(np.exp(y_true), np.exp(y_pred))

mae_scorer = make_scorer(mae_score, greater_is_better=False)


def mean_pred(y_true, y_pred):
    return K.mean(K.abs(K.exp(y_true) - K.exp(y_pred)))

def mean_pred_metrics(y_true, y_pred):
    return K.mean(K.abs(K.exp(y_true) - K.exp(y_pred)))


In [10]:
def hyper_model():
    model = Sequential()
    model.add(Dense(351, input_dim=train_x.shape[1], init='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.578947))
    
    model.add(Dense(293, init='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.26666))
    
    model.add(Dense(46, init='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.188888))
    
    model.add(Dense(1, init='he_normal'))
    model.compile(optimizer='adam',
              loss=mean_pred,metrics=['mae', mean_pred_metrics])
    return model

In [11]:
cv_score = cross_validate_mlp(hyper_model)
("CV score for the final model:", cv_score)

/home/csc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(351, input_dim=1153, kernel_initializer="he_normal")`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/csc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(293, kernel_initializer="he_normal")`
  
/home/csc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(46, kernel_initializer="he_normal")`
  del sys.path[0]
/home/csc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="he_normal")`
/home/csc/anaconda3/lib/python3.6/site-packages/keras/models.py:874: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argu

Train on 100436 samples, validate on 25109 samples
Epoch 1/30
100436/100436 [==============================] - 19s 185us/step - loss: 3225.7151 - mean_absolute_error: 7.0840 - mean_pred_metrics: 3225.7151 - val_loss: 3180.2746 - val_mean_absolute_error: 6.7966 - val_mean_pred_metrics: 3180.2746
Epoch 2/30
100436/100436 [==============================] - 10s 97us/step - loss: 2816.3621 - mean_absolute_error: 4.2023 - mean_pred_metrics: 2816.3621 - val_loss: 2037.0752 - val_mean_absolute_error: 1.1061 - val_mean_pred_metrics: 2037.0752
Epoch 3/30
100436/100436 [==============================] - 10s 99us/step - loss: 2085.1639 - mean_absolute_error: 0.9580 - mean_pred_metrics: 2085.1639 - val_loss: 1654.8122 - val_mean_absolute_error: 0.5962 - val_mean_pred_metrics: 1654.8122
Epoch 4/30
100436/100436 [==============================] - 10s 101us/step - loss: 1967.2538 - mean_absolute_error: 0.8401 - mean_pred_metrics: 1967.2538 - val_loss: 1621.5311 - val_mean_absolute_error: 0.6085 - val_

100436/100436 [==============================] - 9s 92us/step - loss: 1969.1457 - mean_absolute_error: 0.8450 - mean_pred_metrics: 1969.1457 - val_loss: 1578.8081 - val_mean_absolute_error: 0.5782 - val_mean_pred_metrics: 1578.8081
Epoch 5/30
100436/100436 [==============================] - 9s 93us/step - loss: 1904.3249 - mean_absolute_error: 0.7939 - mean_pred_metrics: 1904.3249 - val_loss: 1572.2104 - val_mean_absolute_error: 0.5620 - val_mean_pred_metrics: 1572.2104
Epoch 6/30
100436/100436 [==============================] - 10s 100us/step - loss: 1868.3429 - mean_absolute_error: 0.7580 - mean_pred_metrics: 1868.3429 - val_loss: 1605.6478 - val_mean_absolute_error: 0.5616 - val_mean_pred_metrics: 1605.6478
Epoch 7/30
100436/100436 [==============================] - 10s 97us/step - loss: 1830.5336 - mean_absolute_error: 0.7296 - mean_pred_metrics: 1830.5336 - val_loss: 1467.8644 - val_mean_absolute_error: 0.5176 - val_mean_pred_metrics: 1467.8644
Epoch 8/30
100436/100436 [==========

100436/100436 [==============================] - 9s 94us/step - loss: 1732.2703 - mean_absolute_error: 0.6607 - mean_pred_metrics: 1732.2703 - val_loss: 1339.6569 - val_mean_absolute_error: 0.4383 - val_mean_pred_metrics: 1339.6569
Epoch 8/30
100436/100436 [==============================] - 10s 96us/step - loss: 1690.5047 - mean_absolute_error: 0.6324 - mean_pred_metrics: 1690.5047 - val_loss: 1295.5609 - val_mean_absolute_error: 0.4446 - val_mean_pred_metrics: 1295.5609
Epoch 9/30
100436/100436 [==============================] - 10s 101us/step - loss: 1657.3362 - mean_absolute_error: 0.6106 - mean_pred_metrics: 1657.3362 - val_loss: 1259.1574 - val_mean_absolute_error: 0.4141 - val_mean_pred_metrics: 1259.1574
Epoch 10/30
100436/100436 [==============================] - 10s 101us/step - loss: 1634.9820 - mean_absolute_error: 0.5939 - mean_pred_metrics: 1634.9820 - val_loss: 1315.4953 - val_mean_absolute_error: 0.4390 - val_mean_pred_metrics: 1315.4953
Epoch 11/30
100436/100436 [======

('CV score for the final model:', 0.38607809782451019)

In [ ]:
class NN_with_EntityEmbedding(Model):

    def __init__(self, X_train, y_train, X_val, y_val):
        super().__init__()
        self.nb_epoch = 10
        self.checkpointer = ModelCheckpoint(filepath="best_model_weights.hdf5", verbose=1, save_best_only=True)
        self.max_log_y = max(numpy.max(numpy.log(y_train)), numpy.max(numpy.log(y_val)))
        self.__build_keras_model()
        self.fit(X_train, y_train, X_val, y_val)

    def preprocessing(self, X):
        X_list = split_features(X)
        return X_list

    def __build_keras_model(self):
        models = []

        model_store = Sequential()
        model_store.add(Embedding(1115, 10, input_length=1))
        model_store.add(Reshape(target_shape=(10,)))
        models.append(model_store)

        model_dow = Sequential()
        model_dow.add(Embedding(7, 6, input_length=1))
        model_dow.add(Reshape(target_shape=(6,)))
        models.append(model_dow)

        model_promo = Sequential()
        model_promo.add(Dense(1, input_dim=1))
        models.append(model_promo)

        model_year = Sequential()
        model_year.add(Embedding(3, 2, input_length=1))
        model_year.add(Reshape(target_shape=(2,)))
        models.append(model_year)

        model_month = Sequential()
        model_month.add(Embedding(12, 6, input_length=1))
        model_month.add(Reshape(target_shape=(6,)))
        models.append(model_month)

        model_day = Sequential()
        model_day.add(Embedding(31, 10, input_length=1))
        model_day.add(Reshape(target_shape=(10,)))
        models.append(model_day)

        model_germanstate = Sequential()
        model_germanstate.add(Embedding(12, 6, input_length=1))
        model_germanstate.add(Reshape(target_shape=(6,)))
        models.append(model_germanstate)

        self.model = Sequential()
        self.model.add(Merge(models, mode='concat'))
        self.model.add(Dense(1000, init='uniform'))
        self.model.add(Activation('relu'))
        self.model.add(Dense(500, init='uniform'))
        self.model.add(Activation('relu'))
        self.model.add(Dense(1))
        self.model.add(Activation('sigmoid'))

        self.model.compile(loss='mean_absolute_error', optimizer='adam')

    def _val_for_fit(self, val):
        val = numpy.log(val) / self.max_log_y
        return val

    def _val_for_pred(self, val):
        return numpy.exp(val * self.max_log_y)

    def fit(self, X_train, y_train, X_val, y_val):
        self.model.fit(self.preprocessing(X_train), self._val_for_fit(y_train),
                       validation_data=(self.preprocessing(X_val), self._val_for_fit(y_val)),
                       nb_epoch=self.nb_epoch, batch_size=128,
                       # callbacks=[self.checkpointer],
                       )
        # self.model.load_weights('best_model_weights.hdf5')
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def guess(self, features):
        features = self.preprocessing(features)
        result = self.model.predict(features).flatten()
        return self._val_for_pred(result)


Though this model is not adapted for mere 30 epochs of training, nor for 3-fold CV (I used 5-fold on Kaggle), even though this is a single unbagged model which has been cross-validated on three folds only, we see a very good score:
`CV = 1150` (your score may vary a little).

By the way, this single model, bagged, 5-fold CVed, scored 1116.28 on Kaggle LB.

As we see, this model is considerably better than any other models we had so far. We now take it as the second part of our final ensemble.